In [1]:
from from_root import from_root, from_here
import sys
import os

def from_root(dirname):
    '''Wrapper function for from_root.from_root which, if called will return the root of the dirname asked for'''
    return(os.popen(f'python -c "from from_root import from_root; print(from_root(\'{dirname}\'))"').read().strip())
scripts_path = from_root("scripts")

sys.path.append(scripts_path)
from question_gen import *
from keyword_find import *

## Decide the final answer we are searching for: e.g. Julia Roberts

## Step 1: Extract text about the final answer

### Using wikipedia package

In [34]:
text_wiki = (wikipedia.page("David Tenat", auto_suggest=False)).content

PageError: Page id "David Tenat" does not match any pages. Try another id!

In [33]:
text_wiki

'David John Tennant (né McDonald; born 18 April 1971) is a Scottish actor.\nTennant rose to fame for his role as the tenth incarnation of the Doctor in the BBC sci-fi series Doctor Who (2005–2010). His other notable roles include Giacomo Casanova in the BBC comedy-drama serial Casanova (2005), the title character in the Royal Shakespeare Company\'s film adaptation of Hamlet (2009), Barty Crouch Jr. in the fantasy film Harry Potter and the Goblet of Fire (2005), DI Alec Hardy in the ITV crime drama series Broadchurch (2013–2017), and Kilgrave in the Netflix superhero series Jessica Jones (2015–2019).\nTennant has also worked extensively as a voice actor and on stage, including a portrayal of the title character in a 2008 production of Hamlet and the voice of Scrooge McDuck in DuckTales (2017–2021). In 2015, he received the National Television Award for Special Recognition.\n\n\n== Early life ==\nTennant was born David John McDonald in Bathgate, West Lothian on 18 April 1971, the son of 

In [4]:
text = wiki_autosuggest("Julia Roberts", suggest=False)

In [5]:
text

'Julia Fiona Roberts (born October 28, 1967) is an American actress. One of Hollywood\'s most bankable stars, Roberts is known for her leading roles in films of several genres, from romantic comedies and dramas to thrillers and action films. Many of her films have earned over $100 million at the worldwide box office, and six have ranked among the highest-grossing films of their respective years. Her top-billing films have collectively brought in box office receipts of over $3.8 billion globally. She is the recipient of various accolades, including an Academy Award, a British Academy Film Award, and three Golden Globe Awards. Following an early breakthrough with appearances in Mystic Pizza (1988) and Steel Magnolias (1989), Roberts established herself as a leading actress when she headlined the romantic comedy Pretty Woman (1990), which grossed $464 million worldwide. She went on to star in numerous successful films, including Sleeping with the Enemy (1991), Hook (1991), The Pelican Bri

### using seo blurb functionality

In [12]:
def find_text_seo(input,pages): #e.g input is Las Vegas locations
    '''Take input word adapted for type, 
    return text that keywords can be extracted from'''

    #SEO way
    df = seo.get_serps(input, pages=pages)
    df['link'] = df['link'].str.lower()
    bad_links = ['facebook','twitter','youtube','instagram','linkedin'] #Removing search terms from sites that arnt of interest for keyword extraction
    df = df[~df['link'].str.contains('|'.join(bad_links))]
    #print(df[['title','link']])

    text = str(df['text'].values.tolist())

    #Text Cleaning
    text = re.sub(r"\'", '', text) #Get rid of \'
    text = re.sub(r"\\xa0...", '', text) #Get rid of \\xa0...
    text = re.sub(r"\\n", ' ', text) #Get rid of \\n
    text = re.sub(r"\\u200e", ' ', text) #Get rid of \\u200e
    text = re.sub(r"\"", '', text)
    text = re.sub(r"\'", '', text)
    text = re.sub(r"\“", '', text)
    text = re.sub(r"\”", '', text)
    text = re.sub(r"\[", '', text)
    text = re.sub(r"\]", '', text)
    text = re.sub(r"\)", '', text)
    text = re.sub(r"\(", '', text)
    text = re.sub(r"\•", '', text)
    text = re.sub(r"\▽", '', text)
    text = re.sub(r"\❖", '', text)
    text = re.sub(r"\†", '', text)
    text = re.sub(r"logo", '', text)
    text = re.sub(r"[Vv]iew \d+ more rows", '', text) #Get rid of [Vv]iew \d+ more rows
    text = re.sub(r"\d+ hours ago", '', text)
    text = re.sub(r"[-·—,.;:@#?!$+-]+", ' ', text) 
    text = re.sub(r"U S ", "US ", text)


    text = ' '.join(text.split()) #Single spacing

    return text

In [17]:
results = seo.get_serps("Julia Roberts")
results.iloc[1]['text']

'Julia Roberts, in full Julia Fiona Roberts, (born October 28, 1967, Smyrna, Georgia, U.S.), American actress whose deft performances in varied roles helped\xa0...'

In [15]:
text_seo = find_text_seo("Julia Roberts", 5)
text_seo

'Julia Fiona Roberts born October 28 1967 is an American actress One of Hollywoods most bankable stars Roberts is known for her leading roles in films Julia Roberts in full Julia Fiona Roberts born October 28 1967 Smyrna Georgia US American actress whose deft performances in varied roles helped 18 Apr 2022 Julia Roberts is one of those few actors who have achieved a stardom that never really fades Shes always up there in the pop culture Julia Roberts <p>Roberts new film will be nostalgic for those who grew up on Can Meg Ryan Julia Roberts and Sandra Bullock revive the ailing genre Move over Marvel the meet cute is back – and it feels like slipping 25 Apr 2022 The American actress was born in October 1967 in a small town in Georgia Her acting career has seen great success with her winning three Julia Roberts is a highly acclaimed American actress who has a net worth of 250 million Her film Pretty Woman 1990 established her as a leading star Actress and producer Julia Roberts is the face

# Knowledge Graph

In [27]:
knowledge_graph = KnowledgeGraph()

df_know = knowledge_graph.get_knowledge_graph_df("Julia Roberts")
df_know[['result.description', 'result.@type', 'result.name' , 'result.detailedDescription.url']].iloc[0:4]

,result.description,result.@type,result.name,result.detailedDescription.url
0,American actress,"[Person, Thing]",Julia Roberts,https://en.wikipedia.org/wiki/Julia_Roberts
1,South Korean-American film actress,"[Person, Thing]",Yunjin Kim,https://en.wikipedia.org/wiki/Yunjin_Kim
2,NaN,[Thing],Julia Roberts filmography,https://en.wikipedia.org/wiki/Julia_Roberts_fi...
3,Football player,"[Person, Thing]",Julia Roberts,https://en.wikipedia.org/wiki/Julia_Roberts_(s...


### Knowledge graph returns the sports team 

In [28]:
knowledge_graph = KnowledgeGraph()

df_know = knowledge_graph.get_knowledge_graph_df("France")
df_know[['result.description', 'result.@type', 'result.name' , 'result.detailedDescription.url']]

,result.description,result.@type,result.name,result.detailedDescription.url
0,Football team,"[SportsTeam, Thing]",France national football team,https://en.wikipedia.org/wiki/France_national_...
1,Football league,"[SportsOrganization, Thing, Corporation, Organ...",Ligue 1,https://en.wikipedia.org/wiki/Ligue_1
2,Retail company,"[Corporation, Organization, Thing]",Leroy Merlin,https://en.wikipedia.org/wiki/Leroy_Merlin
3,Telecom company,"[Corporation, Organization, Thing]",Orange S.A.,https://en.wikipedia.org/wiki/Orange_S.A.
4,Air carrier,"[Thing, Corporation, Airline, Organization]",Air France,https://en.wikipedia.org/wiki/Air_France
5,Bank,"[Corporation, Organization, Thing]",Crédit Agricole,https://en.wikipedia.org/wiki/Cr%C3%A9dit_Agri...
6,Meteorological service,"[GovernmentOrganization, Organization, Thing]",Météo-France,https://en.wikipedia.org/wiki/M%C3%A9t%C3%A9o-...
7,Newspaper,"[Periodical, Thing]",Ouest-France,https://en.wikipedia.org/wiki/Ouest-France
8,Radio network,"[Organization, Thing]",France Info,https://en.wikipedia.org/wiki/France_Info_(rad...
9,Company,"[Corporation, Organization, Thing]",Maisons du Monde,https://en.wikipedia.org/wiki/Maisons_du_Monde


## Linkee wikipedia test case

In [3]:
import wikipedia
text_wiki = (wikipedia.page("Tom Hanks", auto_suggest=False)).content

In [28]:
input_text = "Palma"
try:
    text_wiki = (wikipedia.page(input_text, auto_suggest=False)).content
except Exception as e:
    er_var = e
    input_text_page = str(er_var).split('may refer to:')[1].split('\n')[1]
    text_wiki = (wikipedia.page(input_text_page, auto_suggest=False)).content

In [17]:
type(er_var)

wikipedia.exceptions.DisambiguationError

In [18]:
if type(er_var) == wikipedia.exceptions.DisambiguationError:
    print('yes')

yes


In [29]:
text_wiki

"Palma is a town on the northeast coast of Mozambique's Cabo Delgado Province.  Less than 32 kilometres (20 mi) away is the border with Tanzania to the north and north-west.\n\n\n== Economy and languages ==\nVarious languages, including Makwe, Makonde, Mwani, Swahili, and Portuguese, are spoken in the town. The town was historically known for its fishing, basketry, and mat-weaving industries.The local economy was transformed by the discovery of gas in the nearby Indian Ocean in 2010, which led to a rapidly growing liquefied natural gas sector.Palma is close to facilities for natural gas extraction off the coast of Mozambique, a major offshore gas project by Total.\n\n\n== 2021 attack ==\n\nBeginning in October 2017, armed Islamist extremists linked to the Islamic State of Iraq and the Levant (ISIL) launched a jihadist insurgency in the Cabo Delgado region. The militants launched attacks and committed mass beheadings, and in August 2020 seized the port town of Mocimboa da Praia. The gro

In [ ]:
er_var.split(f'{input_text},')